In [49]:
# from utils.segmentation import YOLOSegmentation
import cv2
import numpy as np
import rospy
from cv_bridge import CvBridge, CvBridgeError
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS
from utils.visualisation_utils import depth2pc
from utils.transform_utils import regularize_pc
from utils.ROS_utils import generate_grasps_client, format_pointcloud_msg, run_action
import open3d as o3d
import time

In [50]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [51]:
camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")


Camera topic found


In [52]:
rgb, depth_image, depth_scale = camera.get_rgb_depth()
depth = depth_image * depth_scale

bridge = CvBridge()

pc_fused, pc_colors_fused = depth2pc(depth, intrinsic, rgb)

In [53]:
# cv2.imshow("rgb", rgb)
# cv2.imshow("depth", depth)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [54]:
pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, downsampling_method="voxel", voxel_size=0.005, outlier_filtering_method="radius", radius_param_arg=[25, 0.015])
# pc_fused_, pc_colors_fused_ = regularize_pc(pc_fused, pc_colors_fused, downsampling_method="voxel", voxel_size=0.005, outlier_filtering_method="statistical", statistical_param_arg=[20, 2.0])

Downsampling time: 0.9393620491027832s, Filtering time: 0.06729698181152344s


In [55]:
import open3d as o3d
pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_fused)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_fused/255)
coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
coordinate.translate([0,0 , 0.4])
o3d.visualization.draw_geometries([coordinate, pc_o3d])

In [56]:
pc_fused.shape

(26037, 3)

In [57]:
pc2_msg = format_pointcloud_msg(pc_fused, pc_colors_fused)
bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
bgr_msg = bridge.cv2_to_imgmsg(bgr, encoding="bgr8")

In [75]:
orn, pos, opening, score, detected, detected_with_collision = generate_grasps_client(pc2_msg, bgr_msg)



calling service
